In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import logging

import pandas as pd
import tensorflow.keras as keras

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model

# Log setting
logging.basicConfig(format="%(asctime)s %(levelname)s %(message)s", datefmt="%H:%M:%S", level=logging.INFO)

# Change display.max_rows to show all features.
pd.set_option("display.max_rows", 85)

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers

In [ ]:
training_df = pd.read_csv('/content/drive/MyDrive/train_MachineLearningCVE.csv', skipinitialspace=True)
testing_df = pd.read_csv('/content/drive/MyDrive/test_MachineLearningCVE.csv', skipinitialspace=True)

In [ ]:
print("Training set has {} rows.".format(len(training_df)))
print("Testing set has {} rows.".format(len(testing_df)))

Training set has 2264594 rows.
Testing set has 566149 rows.


In [ ]:
testing_df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,53,180.0,2,2,76,108,38,38,38.000000,0.000000,54,54,54.0,0.000000,1.022222e+06,22222.222220,60.0,1.819341e+01,81.0,49.0,49,4.900000e+01,0.0,49,49.0,50,50.000000,0.00000,50,50,0,0,0,0,64.0,64.0,11111.111110,11111.111110,38,54,44.400000,8.763561,7.680000e+01,0,0,0,0,0,0,0,0,1,55.500000,38.000000,54.0,64.0,0,0,0,0,0,0,2,76,2,108,0.0,0.0,1,32.0,0.0,0.0,0,0,0.0,0.0,0,0,0
1,80,84213354.0,10,6,1077,11595,357,0,107.700000,172.043308,5792,0,1932.5,2362.781391,1.504749e+02,0.189994,5614223.6,2.160000e+07,83600000.0,1.0,84100000,9.340625e+06,27900000.0,83600000,1.0,582565,116513.000000,188993.72180,434860,66,0,0,0,0,316.0,200.0,0.118746,0.071248,0,5792,745.411765,1605.800854,2.578596e+06,0,0,0,0,1,0,0,0,0,792.000000,107.700000,1932.5,316.0,0,0,0,0,0,0,10,1077,6,11595,274.0,235.0,4,20.0,999.0,0.0,999,999,83600000.0,0.0,83600000,83600000,4
2,88,156.0,1,4,0,12,0,0,0.000000,0.000000,6,0,3.0,3.464102,7.692308e+04,32051.282051,39.0,7.666812e+01,154.0,0.0,0,0.000000e+00,0.0,0,0.0,2,0.666667,0.57735,1,0,0,0,0,0,32.0,104.0,6410.256410,25641.025641,0,6,2.000000,3.098387,9.600000e+00,0,0,0,0,1,1,0,0,4,2.400000,0.000000,3.0,32.0,0,0,0,0,0,0,1,0,4,12,254.0,0.0,0,32.0,0.0,0.0,0,0,0.0,0.0,0,0,0
3,53,71031.0,1,1,47,132,47,47,47.000000,0.000000,132,132,132.0,0.000000,2.520026e+03,28.156720,71031.0,0.000000e+00,71031.0,71031.0,0,0.000000e+00,0.0,0,0.0,0,0.000000,0.00000,0,0,0,0,0,0,20.0,20.0,14.078360,14.078360,47,132,75.333333,49.074773,2.408333e+03,0,0,0,0,0,0,0,0,1,113.000000,47.000000,132.0,20.0,0,0,0,0,0,0,1,47,1,132,0.0,0.0,0,20.0,0.0,0.0,0,0,0.0,0.0,0,0,0
4,80,84910301.0,6,5,400,11595,400,0,66.666667,163.299316,8688,0,2319.0,3774.237539,1.412667e+02,0.129548,8491030.1,2.680000e+07,84800000.0,1.0,84800000,1.700000e+07,37900000.0,84800000,1.0,149673,37418.250000,65560.01703,135271,185,0,0,0,0,200.0,168.0,0.070663,0.058886,0,8688,999.583333,2559.088597,6.548934e+06,0,0,0,0,1,0,0,0,0,1090.454545,66.666667,2319.0,200.0,0,0,0,0,0,0,6,400,5,11595,251.0,235.0,1,32.0,4.0,0.0,4,4,84800000.0,0.0,84800000,84800000,4


In [ ]:

# Our new labels
classes=["Normal","Benign","FTP-Patator","SSH-Patator","DoS slowloris","DoS Slowhttptest","DoS Hulk","DoS GoldenEye","Heartbleed","Web Attack–Brute force","Web Attack–XSS","Web Attack–SQL Injection","Infiltration","Bot","DDoS","Port scan"]

#Helper function to label samples to 5 classes
def label_attack (row):
    if row["Label"] == 0:
        return classes[1]
    if row["Label"] == 1:
        return classes[2]
    if row["Label"] == 2:
        return classes[3]
    if row["Label"] == 3:
        return classes[4]
    if row["Label"] == 4:
        return classes[5]
    if row["Label"] == 4:
        return classes[5]
    if row["Label"] == 5:
        return classes[6]
    if row["Label"] == 6:
        return classes[7]
    if row["Label"] == 7:
        return classes[8]
    if row["Label"] == 8:
        return classes[9]
    if row["Label"] == 9:
        return classes[10]
    if row["Label"] == 10:
        return classes[11]
    if row["Label"] == 11:
        return classes[12]
    if row["Label"] == 12:
        return classes[13]
    if row["Label"] == 13:
        return classes[14]
    if row["Label"] == 12:
        return classes[13]
    if row["Label"] == 13:
        return classes[14]
    if row["Label"] == 14:
        return classes[15]
    return classes[0]




In [ ]:
test_samples_length = len(testing_df)
df=pd.concat([training_df,testing_df])
df["Class"]=df.apply(label_attack,axis=1)

In [ ]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Class
0,80,6018089.0,5,3,177,994,159,0,35.400000,69.143329,988,0,331.333333,568.697928,194.580040,1.329326,8.597270e+05,2.154101e+06,5742166.0,12.0,6018089,1.504522e+06,2.827010e+06,5742166,66.0,39004,19502.0,2.689127e+04,38517,487,0,0,0,0,112.0,72.0,0.830829,0.498497,0,988,130.111111,325.808396,106151.111100,0,0,0,1,0,0,0,0,0,146.375000,35.400000,331.333333,112.0,0,0,0,0,0,0,5,177,3,994,8192.0,946.0,4,20.0,275923.0000,0.000,275923,275923,5742166.0,0.000000,5742166,5742166,0,Benign
1,443,323049.0,8,6,531,3208,194,0,66.375000,85.880211,1460,0,534.666667,710.341233,11574.095570,43.337079,2.484992e+04,2.474712e+04,67171.0,3.0,323049,4.614986e+04,3.173143e+04,82654,3.0,241830,48366.0,4.398387e+04,90690,192,0,0,0,0,172.0,132.0,24.764045,18.573034,0,1460,249.266667,492.297742,242357.066700,0,0,0,1,0,0,0,0,0,267.071429,66.375000,534.666667,172.0,0,0,0,0,0,0,8,531,6,3208,8192.0,980.0,4,20.0,0.0000,0.000,0,0,0.0,0.000000,0,0,0,Benign
2,80,39270118.0,9,10,898,3944,431,0,99.777778,187.795619,1963,0,394.400000,721.308410,123.299859,0.483828,2.181673e+06,4.150573e+06,10000000.0,68.0,30400000,3.796280e+06,5.157701e+06,10000000,357.0,39200000,4359251.0,5.120822e+06,10000000,174,0,0,0,0,192.0,248.0,0.229182,0.254647,0,1963,242.100000,534.965606,286188.200000,0,0,0,1,0,0,0,0,1,254.842105,99.777778,394.400000,192.0,0,0,0,0,0,0,9,898,10,3944,8192.0,980.0,8,20.0,123253.6667,149943.928,296394,36667,10000000.0,300.561142,10000000,9999980,0,Benign
3,4848,43.0,1,1,2,6,2,2,2.000000,0.000000,6,6,6.000000,0.000000,186046.511600,46511.627910,4.300000e+01,0.000000e+00,43.0,43.0,0,0.000000e+00,0.000000e+00,0,0.0,0,0.0,0.000000e+00,0,0,0,0,0,0,24.0,20.0,23255.813950,23255.813950,2,6,3.333333,2.309401,5.333333,0,0,0,1,0,0,0,0,1,5.000000,2.000000,6.000000,24.0,0,0,0,0,0,0,1,2,1,6,1024.0,0.0,0,24.0,0.0000,0.000,0,0,0.0,0.000000,0,0,10,Web Attack–SQL Injection
4,80,5754816.0,3,1,12,0,6,0,4.000000,3.464102,0,0,0.000000,0.000000,2.085210,0.695070,1.918272e+06,3.294694e+06,5722612.0,45.0,5754816,2.877408e+06,4.023726e+06,5722612,32204.0,0,0.0,0.000000e+00,0,0,0,0,0,0,72.0,32.0,0.521303,0.173768,0,6,2.400000,3.286335,10.800000,0,0,0,1,0,0,0,0,0,3.000000,4.000000,0.000000,72.0,0,0,0,0,0,0,3,12,1,0,8192.0,29200.0,2,20.0,0.0000,0.000,0,0,0.0,0.000000,0,0,0,Benign


In [ ]:
df=df.drop("Label",axis=1)

In [ ]:
print(df['Class'].unique())

['Benign' 'Web Attack–SQL Injection' 'DoS Slowhttptest' 'Heartbleed'
 'DoS slowloris' 'DoS Hulk' 'DoS GoldenEye' 'Infiltration' 'FTP-Patator'
 'Bot' 'Port scan' 'Web Attack–XSS' 'Web Attack–Brute force' 'DDoS'
 'SSH-Patator']


In [ ]:
training_df= df.iloc[:-test_samples_length, :]
testing_df= df.iloc[-test_samples_length:,:]

In [ ]:
training_outcomes=training_df["Class"].unique()
testing_outcomes=testing_df["Class"].unique()
print("The training set has {} possible outcomes \n".format(len(training_outcomes)) )
print(", ".join(training_outcomes)+".")
print("\nThe testing set has {} possible outcomes \n".format(len(testing_outcomes)))
print(", ".join(testing_outcomes)+".")

The training set has 15 possible outcomes 

Benign, Web Attack–SQL Injection, DoS Slowhttptest, Heartbleed, DoS slowloris, DoS Hulk, DoS GoldenEye, Infiltration, FTP-Patator, Bot, Port scan, Web Attack–XSS, Web Attack–Brute force, DDoS, SSH-Patator.

The testing set has 15 possible outcomes 

Benign, DoS Slowhttptest, Web Attack–SQL Injection, DoS slowloris, Heartbleed, DoS GoldenEye, DoS Hulk, Infiltration, Bot, FTP-Patator, Port scan, Web Attack–XSS, Web Attack–Brute force, DDoS, SSH-Patator.


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.layers import Input,Dropout,Dense
from keras.models import Model
from keras import regularizers
from keras.utils.data_utils import get_file
%matplotlib inline

In [ ]:
# Helper function for scaling continous values
def minmax_scale_values(training_df,testing_df, col_name):
    scaler = MinMaxScaler()
    scaler = scaler.fit(training_df[col_name].values.reshape(-1, 1))
    train_values_standardized = scaler.transform(training_df[col_name].values.reshape(-1, 1))
    training_df[col_name] = train_values_standardized
    test_values_standardized = scaler.transform(testing_df[col_name].values.reshape(-1, 1))
    testing_df[col_name] = test_values_standardized
    
    
#Helper function for one hot encoding
def encode_text(training_df,testing_df, name):
    training_set_dummies = pd.get_dummies(training_df[name])
    testing_set_dummies = pd.get_dummies(testing_df[name])
    for x in training_set_dummies.columns:
        dummy_name = "{}_{}".format(name, x)
        training_df[dummy_name] = training_set_dummies[x]
        if x in testing_set_dummies.columns :
            testing_df[dummy_name]=testing_set_dummies[x]
        else :
            testing_df[dummy_name]=np.zeros(len(testing_df))
    training_df.drop(name, axis=1, inplace=True)
    testing_df.drop(name, axis=1, inplace=True)
    
    
sympolic_columns=["protocol_type","service","flag"]
label_column="Class"
for column in df.columns :
    if column in sympolic_columns:
        encode_text(training_df,testing_df,column)
    elif not column == label_column:
        minmax_scale_values(training_df,testing_df, column)

In [ ]:
training_df.head(5)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Class
0,0.001221,5.015074e-02,0.000018,0.000010,1.372093e-05,1.516508e-06,0.006406,0.00000,0.005959,0.009808,0.050589,0.000000,0.057122,0.069399,9.395463e-08,3.281561e-07,7.164392e-03,0.025402,4.785138e-02,1.000000e-07,0.050151,0.012538,0.033415,0.047851,5.500000e-07,0.000325,0.000163,0.000319,0.000321,0.000004,0.0,0.0,0.0,0.0,0.000024,0.000012,2.769428e-07,2.492486e-07,0.000000,0.039807,0.038989,0.068859,4.738889e-03,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00000,0.037596,0.005959,0.057122,0.000024,0.0,0.0,0.0,0.0,0.0,0.0,0.000018,1.375255e-05,0.000010,1.516508e-06,0.125002,0.014435,0.000019,0.144928,0.002508,0.000000,0.002508,0.002508,0.047851,0.000000,0.047851,0.047851,Benign
1,0.006760,2.692075e-03,0.000032,0.000021,4.116279e-05,4.894325e-06,0.007816,0.00000,0.011173,0.012183,0.074757,0.000000,0.092176,0.086683,5.588651e-06,1.083009e-05,2.070827e-04,0.000292,5.597583e-04,2.500000e-08,0.002692,0.000385,0.000375,0.000689,2.500000e-08,0.002015,0.000403,0.000521,0.000756,0.000002,0.0,0.0,0.0,0.0,0.000037,0.000023,8.254682e-06,9.286517e-06,0.000000,0.058824,0.074695,0.104046,1.081951e-02,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00000,0.068597,0.011173,0.092176,0.000037,0.0,0.0,0.0,0.0,0.0,0.0,0.000032,4.125766e-05,0.000021,4.894325e-06,0.125002,0.014954,0.000019,0.144928,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign
2,0.001221,3.272510e-01,0.000036,0.000034,6.961240e-05,6.017212e-06,0.017365,0.00000,0.016795,0.026640,0.100512,0.000000,0.067994,0.088022,5.953639e-08,1.167818e-07,1.818061e-02,0.048945,8.333333e-02,5.666667e-07,0.253333,0.031636,0.060964,0.083333,2.975000e-06,0.326667,0.036327,0.060660,0.083333,0.000001,0.0,0.0,0.0,0.0,0.000041,0.000042,7.639396e-08,1.273233e-07,0.000000,0.079089,0.072547,0.113064,1.277626e-02,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00641,0.065456,0.016795,0.067994,0.000041,0.0,0.0,0.0,0.0,0.0,0.0,0.000036,6.977284e-05,0.000034,6.017212e-06,0.125002,0.014954,0.000037,0.144928,0.001120,0.002021,0.002694,0.000333,0.083333,0.000004,0.083333,0.083333,Benign
3,0.073976,3.583333e-07,0.000000,0.000003,1.550388e-07,9.153974e-09,0.000081,0.00086,0.000337,0.000000,0.000307,0.002072,0.001034,0.000000,8.983414e-05,1.162790e-02,3.583333e-07,0.000000,3.583333e-07,3.583333e-07,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000005,0.000003,7.751938e-03,1.162791e-02,0.001381,0.000242,0.000999,0.000488,2.380952e-07,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00641,0.001284,0.000337,0.001034,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.553961e-07,0.000003,9.153974e-09,0.015625,0.000000,0.000000,0.173913,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [ ]:
x,y=training_df,training_df.pop("Class").values
x=x.values
x_test,y_test=testing_df,testing_df.pop("Class").values
x_test=x_test.values
y0=np.ones(len(y),np.int8)
y0[np.where(y==classes[0])]=0
y0_test=np.ones(len(y_test),np.int8)
y0_test[np.where(y_test==classes[0])]=0

In [ ]:
x_test.shape

(566149, 78)

In [ ]:
def getModel1():
    inp = Input(shape=(x.shape[1],))
    d1=Dropout(0)(inp)
    encoded = Dense(8, activation='relu', activity_regularizer=regularizers.l2(10e-5))(d1)
    decoded = Dense(x.shape[1], activation='relu')(encoded)
    single_autoencoder = Model(inp, decoded)
    single_autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    return single_autoencoder

In [ ]:
single_autoencoder=getModel1()
history1=single_autoencoder.fit(x[np.where(y0==1)],x[np.where(y0==1)],
               epochs=10,
                batch_size=100,
                shuffle=True,
                validation_split=0.1
                       )

Epoch 1/10
20382/20382 [==============================] - 39s 2ms/step - loss: 0.0037 - val_loss: 0.0041
Epoch 2/10
20382/20382 [==============================] - 38s 2ms/step - loss: 0.0035 - val_loss: 0.0039
Epoch 3/10
20382/20382 [==============================] - 38s 2ms/step - loss: 0.0034 - val_loss: 0.0039
Epoch 4/10
20382/20382 [==============================] - 38s 2ms/step - loss: 0.0034 - val_loss: 0.0039
Epoch 5/10
20382/20382 [==============================] - 38s 2ms/step - loss: 0.0034 - val_loss: 0.0039
Epoch 6/10
20382/20382 [==============================] - 38s 2ms/step - loss: 0.0034 - val_loss: 0.0038
Epoch 7/10
20382/20382 [==============================] - 38s 2ms/step - loss: 0.0034 - val_loss: 0.0039
Epoch 8/10
20382/20382 [==============================] - 37s 2ms/step - loss: 0.0034 - val_loss: 0.0038
Epoch 9/10
20382/20382 [==============================] - 37s 2ms/step - loss: 0.0034 - val_loss: 0.0038
Epoch 10/10
20382/20382 [==============================

In [ ]:
# Helper function that calculates the reconstruction loss of each data sample
def calculate_losses(x,preds):
    losses=np.zeros(len(x))
    for i in range(len(x)):
        losses[i]=((preds[i] - x[i]) ** 2).mean(axis=None)
    return losses

# We set the threshold equal to the training loss of the autoencoder
threshold=history1.history["loss"][-1]

testing_set_predictions=single_autoencoder.predict(x_test)
test_losses=calculate_losses(x_test,testing_set_predictions)
testing_set_predictions=np.zeros(len(test_losses))
testing_set_predictions[np.where(test_losses>threshold)]=1

In [ ]:
accuracy=accuracy_score(y0_test,testing_set_predictions)
recall=recall_score(y0_test,testing_set_predictions)
precision=precision_score(y0_test,testing_set_predictions)
f1=f1_score(y0_test,testing_set_predictions)
print("Performance over the testing data set \n")
print("Accuracy : {} , Recall : {} , Precision : {} , F1 : {}\n".format(1-accuracy,1-recall,1-precision,1-f1 ))
#for class_ in classes:
    #print(class_+" Detection Rate : {}".format(len(np.where(np.logical_and(testing_set_predictions==1 , y_test==class_))[0])/len(np.where(y_test==class_)[0])))

Performance over the testing data set 

Accuracy : 0.771328749145543 , Recall : 0.771328749145543 , Precision : 0.0 , F1 : 0.6277747189161758

